In [3]:
import requests
import time
import pandas as pd
import os

TOKEN = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
BASE_URL = "https://ows.goszakup.gov.kz"
API_ENDPOINT = "/v3/plans"
parquet_file = "D:/qbs/goszakup_plans_customer_list_data.parquet"

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

# 1. Загружаем существующие данные (если есть)
if os.path.exists(parquet_file):
    df_existing = pd.read_parquet(parquet_file, engine="pyarrow")
    seen_ids = set(df_existing["pid"])
    print(f"Загружено ранее: {len(df_existing)} записей")
else:
    df_existing = pd.DataFrame()
    seen_ids = set()

# 2. Начинаем загрузку без search_after (чтобы получить самые свежие записи)
params = {"limit": 50}
all_data = []
next_page = API_ENDPOINT + "?limit=50"

while next_page:
    try:
        response = requests.get(BASE_URL + next_page, headers=headers, timeout=30)
        response.raise_for_status()
        data = response.json()

        items = data.get("items", [])
        if not items:
            print("Новых данных нет, загрузка завершена.")
            break

        # 3. Фильтруем дубликаты
        new_items = [item for item in items if item["pid"] not in seen_ids]
        if not new_items:
            print("API вернуло только дубликаты, остановка загрузки.")
            break

        all_data.extend(new_items)
        seen_ids.update(item["id"] for item in new_items)

        print(f"Загружено {len(all_data)} новых записей...")

        # 4. Получаем `next_page`
        next_page = data.get("next_page")
        if not next_page:
            print("Достигнута последняя страница, завершаем загрузку.")
            break

        # 5. Промежуточное сохранение каждые 500 записей
        if len(all_data) >= 500:
            df_new = pd.DataFrame(all_data)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
            df_combined.to_parquet(parquet_file, index=False, engine="pyarrow")
            print("Промежуточное сохранение...")
            all_data = []  # Очищаем список после сохранения

        time.sleep(5)

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        print("Ждём 30 сек и пробуем снова...")
        time.sleep(30)

# 6. Финальное сохранение
if all_data:
    df_new = pd.DataFrame(all_data)
    df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    df_combined.to_parquet(parquet_file, index=False, engine="pyarrow")
    print(f"Дозагрузка завершена, новые данные добавлены в {parquet_file}")
else:
    print("Новых данных не найдено.")

Загружено ранее: 2785 записей
API вернуло только дубликаты, остановка загрузки.
Новых данных не найдено.


In [4]:
df = pd.read_parquet("D:/qbs/goszakup_plans_customer_list_data.parquet")
df.head()

,pid,bin,name_ru,name_kz,doc_count
0,171777,161240020739,"Товарищество с ограниченной ответственностью ""...","""XAMID 001"" жауапкершілігі шектеулі серіктестігі",0
1,241268,010440001806,"ТОО ""ЭНКО-01""","""ЭНКО-01"" ЖШС",0
2,54126,031040007024,"РГУ ""Войсковая часть 01098"" Министерства оборо...",Қазақстан Республикасы Қорғаныс министрлігінің...,516
3,52017,020440006786,"РГУ ""Войсковая часть 01731"" Министерства оборо...",Қазақстан Республикасы Қорғаныс министрлігінің...,2915
4,105139,990240008066,"Республиканское государственное учреждение ""Во...",Қазақстан Республикасы Қорғаныс министрлігінің...,85


In [5]:
import numpy as np
df = df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)

# Check for duplicates
duplicates = df.duplicated()
print(duplicates)

duplicates_count = df.duplicated().sum()
print(f'Количество дубликатов: {duplicates_count}')

0       False
1       False
2       False
3       False
4       False
        ...  
2780    False
2781    False
2782    False
2783    False
2784    False
Length: 2785, dtype: bool
Количество дубликатов: 0


C:\Users\alena\AppData\Local\Temp\ipykernel_1372\1402623328.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: tuple(x) if isinstance(x, np.ndarray) else x)
